In [1]:
#@title 1.3. IMPORT LIBRARY
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
#@title 1.4. RETRIEVE DATA
df_credit = pd.read_csv('/workspaces/functional-kan/phase_1/data/german_credit_data.csv')
df_credit['Risk'] = df_credit['Risk'].map({'good': 1, 'bad': 0})
df_credit.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,1
1,22,female,2,own,little,moderate,5951,48,radio/TV,0
2,49,male,1,own,little,NaN,2096,12,education,1
3,45,male,2,free,little,little,7882,42,furniture/equipment,1
4,53,male,2,free,little,little,4870,24,car,0


In [3]:
X = df_credit.drop(columns='Risk')
X = pd.get_dummies(X)
y = df_credit['Risk']

,Age,Job,Credit amount,Duration,Sex_female,Sex_male,Housing_free,Housing_own,Housing_rent,Saving accounts_little,...,Checking account_moderate,Checking account_rich,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others
0,67,2,1169,6,False,True,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
1,22,2,5951,48,True,False,False,True,False,True,...,True,False,False,False,False,False,False,True,False,False
2,49,1,2096,12,False,True,False,True,False,True,...,False,False,False,False,False,True,False,False,False,False
3,45,2,7882,42,False,True,True,False,False,True,...,False,False,False,False,False,False,True,False,False,False
4,53,2,4870,24,False,True,True,False,False,True,...,False,False,False,True,False,False,False,False,False,False


In [4]:
#@title 4.2. MLP PREDICTION TRIAL
n_simulation = 50
test_size = 0.3
total = 0

for idx in tqdm(range(n_simulation)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y, random_state=idx)

    mlp = GaussianNB()
    mlp.fit(X_train, y_train)

    y_pred = mlp.predict(X_test)
    total += (y_pred == y_test).mean()

accuracy = total / n_simulation
accuracy

100%|██████████| 50/50 [00:00<00:00, 224.44it/s]


np.float64(0.6747333333333333)

In [5]:
from kan import *
torch.set_default_dtype(torch.float64)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# create a KAN: 2D inputs, 1D output, and 5 hidden neurons. cubic spline (k=3), 5 grid intervals (grid=5).
model = KAN(width=[5,3,1], grid=3, k=3, seed=42, device=device)

cpu
checkpoint directory created: ./model
saving model version 0.0


In [6]:
dataset = dict()
dataset['train_input'] = torch.from_numpy(X_train.values)
dataset['train_label'] = torch.from_numpy(y_train.values)

dataset['test_input'] = torch.from_numpy(X_test.values)
dataset['test_label'] = torch.from_numpy(y_test.values)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.